---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

# Assignment 3 - Evaluation

datasets:
https://www.kaggle.com/dalpozz/creditcardfraud.

In [1]:
import numpy as np
import pandas as pd

Percentage of the observations in the dataset that is instances of fraud.

In [2]:
def answer_one():
    
    # Your code here
    df = pd.read_csv('fraud_data.csv')
    percentage = (len(df[df['Class'] == 1]) / len(df[df['Class'] == 0]))
    return percentage

# answer_one()

0.9835891762319642

In [3]:
# Use X_train, X_test, y_train, y_test for all of the following questions
from sklearn.model_selection import train_test_split

df = pd.read_csv('fraud_data.csv')

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [4]:
def answer_two():
    from sklearn.dummy import DummyClassifier
    from sklearn.metrics import recall_score, accuracy_score
    
    # Your code here
    dummy_majority = DummyClassifier(strategy = 'most_frequent').fit(X_train, y_train)
    y_dummy_predictions = dummy_majority.predict(X_test)
    
    res = (accuracy_score(y_test, y_dummy_predictions), recall_score(y_test, y_dummy_predictions))
    
    return res

# answer_two()

(0.98525073746312686, 0.0)

In [5]:
def answer_three():
    from sklearn.metrics import recall_score, precision_score, accuracy_score
    from sklearn.svm import SVC

    # Your code here
    model = SVC().fit(X_train, y_train)
    y_predictions = model.predict(X_test)
    res = (accuracy_score(y_test, y_predictions), recall_score(y_test, y_predictions), precision_score(y_test, y_predictions))

    return res

# answer_three()

(0.99078171091445433, 0.375, 1.0)

In [6]:
def answer_four():
    from sklearn.metrics import confusion_matrix
    from sklearn.svm import SVC

    # Your code here
    model = SVC(C = 1e9, gamma = 1e-07).fit(X_train, y_train)
    y_predictions = model.decision_function(X_test) > -220
    
    res = confusion_matrix(y_test, y_predictions)
    
    return res

# answer_four()

array([[5320,   24],
       [  14,   66]])

In [7]:
def answer_five():
        
    # Your code here
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_curve
    from sklearn.metrics import precision_recall_curve
    
    model = LogisticRegression().fit(X_train, y_train)
    
    y_scores_lr = model.fit(X_train, y_train).decision_function(X_test)
    
    # precision_recall_curve
    precision, recall, thresholds = precision_recall_curve(y_test, y_scores_lr)
    closest_zero_p = np.argmin(np.abs(precision-0.75))
    closest_zero_r = recall[closest_zero_p]
    
    # roc_curve
    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_scores_lr)
    closest_zero_fpr_lr = np.argmin(np.abs(fpr_lr - 0.16))
    closest_zero_tpr_lr = recall[closest_zero_fpr_lr]
    
    res = (closest_zero_r, closest_zero_tpr_lr)
    return res

# answer_five()

(0.82499999999999996, 0.98750000000000004)

In [8]:
def answer_six():    
    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LogisticRegression

    # Your code here
    model = LogisticRegression()

    grid_values = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

    grid_lr_r = GridSearchCV(model, param_grid = grid_values, scoring = 'recall')
    grid_lr_r.fit(X_train, y_train)
    
    res = np.array(grid_lr_r.cv_results_['mean_test_score'].reshape(5,2))
    return res

# answer_six()

array([[ 0.66666667,  0.76086957],
       [ 0.80072464,  0.80434783],
       [ 0.8115942 ,  0.8115942 ],
       [ 0.80797101,  0.8115942 ],
       [ 0.80797101,  0.80797101]])

In [9]:
# visualize results from the grid search
def GridSearch_Heatmap(scores):
#     %matplotlib notebook
    import seaborn as sns
    import matplotlib.pyplot as plt
    plt.figure()
    sns.heatmap(scores.reshape(5,2), xticklabels=['l1','l2'], yticklabels=[0.01, 0.1, 1, 10, 100])
    plt.yticks(rotation=0);

# GridSearch_Heatmap(answer_six())